In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD, Adam
from keras.constraints import maxnorm
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

Using TensorFlow backend.


In [2]:
# convert series to supervised learning & normalize input variables
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [3]:
# Function to create model, required for KerasClassifier

#1. Batch Size and Epochs Model
def create_model_1():
    
    # create model
    model = Sequential()
    model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    return model

#2. Training Optimization Model
def create_model_2(optimizer='adam'):
    
    # create model
    model = Sequential()
    model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer=optimizer, metrics=["accuracy"])
    
    return model

#3. Learning Rate and Momentum Model
def create_model_3(learn_rate=0.01, momentum=0):
    
    # create model
    model = Sequential()
    model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='mae', optimizer=optimizer, metrics=["accuracy"])
    
    return model

#4. Network Weight Initialization Model
def create_model_4(init_mode='uniform'):
    
    # create model
    model = Sequential()
    model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2]), kernel_initializer=init_mode))
    model.add(Dense(1, kernel_initializer=init_mode))
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    
    return model

#5. Neuron Activation Function Model
def create_model_5(activation='relu'):
    # create model
    model = Sequential()
    model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    
    return model

#6. Dropout Regularization Model
def create_model_6(dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(LSTM(64, input_shape=(train_X.shape[1], train_X.shape[2]), kernel_constraint=maxnorm(weight_constraint))) 
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    
    return model

#7. Number of Neurons in the Hidden Layer
def create_model_7(neurons=1):
    # create model
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(train_X.shape[1], train_X.shape[2]), kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
                  
    return model

In [4]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
#dataset = numpy.loadtxt("./reports/pima-indians-diabetes.data.csv", delimiter=",")
dataset = read_csv('../reports/company_report_2.csv', header=0, index_col="time")
dataset = dataset[['volume_tests', 'date', 'month', 'is_weekend', 'quality_too_poor', 'number_busy', 'temporarily_unable_test', 'outage_hrs', 'number_test_types', 'numbers_tested', 'min_commit']]
dataset = dataset.dropna(axis='columns')
values = dataset.values

# specify the number of days and features 
n_days = 7
n_features = dataset.shape[1]

# integer encode direction
# encoder = LabelEncoder()
# values[:,3] = encoder.fit_transform(values[:,3])
# print(values)

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# split into input (X) and output (Y) variables
# X = values[:,0:dataset.shape[1]-1]
# Y = values[:,0]

# frame as supervised learning
reframed = series_to_supervised(scaled, n_days, 1)
print(reframed.shape)

(1202, 88)


In [5]:
# split into train and test sets
values = reframed.values

train_size = len(values)
train = values

# split into input and outputs
n_obs = n_days * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
print(train_X.shape, train_y.shape)

(1202, 77) 1202 (1202,)
(1202, 7, 11) (1202,)


## 1. Tune Batch Size and Number of Epochs

In [6]:
# create model
model = KerasClassifier(build_fn=create_model_1, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]# summarize results
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(train_X, train_y)

In [7]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.042324 using {'batch_size': 10, 'epochs': 10}
0.042324 (0.084647) with: {'batch_size': 10, 'epochs': 10}
0.042324 (0.084647) with: {'batch_size': 10, 'epochs': 30}
0.042324 (0.084647) with: {'batch_size': 10, 'epochs': 50}
0.042324 (0.084647) with: {'batch_size': 20, 'epochs': 10}
0.042324 (0.084647) with: {'batch_size': 20, 'epochs': 30}
0.042324 (0.084647) with: {'batch_size': 20, 'epochs': 50}
0.042324 (0.084647) with: {'batch_size': 40, 'epochs': 10}
0.042324 (0.084647) with: {'batch_size': 40, 'epochs': 30}
0.042324 (0.084647) with: {'batch_size': 40, 'epochs': 50}
0.042324 (0.084647) with: {'batch_size': 60, 'epochs': 10}
0.042324 (0.084647) with: {'batch_size': 60, 'epochs': 30}
0.042324 (0.084647) with: {'batch_size': 60, 'epochs': 50}
0.042324 (0.084647) with: {'batch_size': 80, 'epochs': 10}
0.042324 (0.084647) with: {'batch_size': 80, 'epochs': 30}
0.042324 (0.084647) with: {'batch_size': 80, 'epochs': 50}
0.042324 (0.084647) with: {'batch_size': 100, 'epochs': 10}
0

## 2. Tune Training Optimization Algorithm

In [8]:
# create model
model = KerasClassifier(build_fn=create_model_2, epochs=10, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_X, train_y)

In [9]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.016625 using {'optimizer': 'SGD'}
0.016625 (0.023511) with: {'optimizer': 'SGD'}
0.016625 (0.023511) with: {'optimizer': 'RMSprop'}
0.016625 (0.023511) with: {'optimizer': 'Adagrad'}
0.000831 (0.001176) with: {'optimizer': 'Adadelta'}
0.016625 (0.023511) with: {'optimizer': 'Adam'}
0.016625 (0.023511) with: {'optimizer': 'Adamax'}
0.016625 (0.023511) with: {'optimizer': 'Nadam'}


## 3. Tune Learning Rate and Momentum

In [10]:
# create model
model = KerasClassifier(build_fn=create_model_3, epochs=10, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_X, train_y)

/home/shanebowen/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [11]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.016625 using {'learn_rate': 0.001, 'momentum': 0.0}
0.016625 (0.023511) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.016625 (0.023511) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.016625 (0.023511) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.016625 (0.023511) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.016625 (0.023511) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.016625 (0.023511) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.016625 (0.023511) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.016625 (0.023511) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.016625 (0.023511) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.016625 (0.023511) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.016625 (0.023511) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.016625 (0.023511) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.016625 (0.023511) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.016625 (0.023511) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.016625 (0.023511) with: {'learn_rate':

## 4. Tune Network Weight Initialization

In [12]:
# create model
model = KerasClassifier(build_fn=create_model_4, epochs=10, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_X, train_y)

/home/shanebowen/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [13]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.016625 using {'init_mode': 'uniform'}
0.016625 (0.023511) with: {'init_mode': 'uniform'}
0.016625 (0.023511) with: {'init_mode': 'lecun_uniform'}
0.016625 (0.023511) with: {'init_mode': 'normal'}
0.016625 (0.023511) with: {'init_mode': 'zero'}
0.016625 (0.023511) with: {'init_mode': 'glorot_normal'}
0.016625 (0.023511) with: {'init_mode': 'glorot_uniform'}
0.016625 (0.023511) with: {'init_mode': 'he_normal'}
0.016625 (0.023511) with: {'init_mode': 'he_uniform'}


## 5. Tune Neuron Activation Function

In [14]:
# create model
model = KerasClassifier(build_fn=create_model_5, epochs=10, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_X, train_y)

/home/shanebowen/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [15]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.016625 using {'activation': 'softmax'}
0.016625 (0.023511) with: {'activation': 'softmax'}
0.016625 (0.023511) with: {'activation': 'softplus'}
0.016625 (0.023511) with: {'activation': 'softsign'}
0.016625 (0.023511) with: {'activation': 'relu'}
0.016625 (0.023511) with: {'activation': 'tanh'}
0.016625 (0.023511) with: {'activation': 'sigmoid'}
0.016625 (0.023511) with: {'activation': 'hard_sigmoid'}
0.016625 (0.023511) with: {'activation': 'linear'}


## 6. Tune Dropout Regularization

In [18]:
# create model
model = KerasClassifier(build_fn=create_model_6, epochs=10, batch_size=10, verbose=0)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_X, train_y)

/home/shanebowen/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RuntimeError: Variable *= value not supported. Use `var.assign(var * value)` to modify the variable or `var = var * value` to get a new Tensor object.

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## 7. Tune Number of Neurons in the Hidden Layer

In [17]:
# create model
model = KerasClassifier(build_fn=create_model_7, epochs=10, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [8, 16, 32, 64, 128]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_X, train_y)

/home/shanebowen/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RuntimeError: Variable *= value not supported. Use `var.assign(var * value)` to modify the variable or `var = var * value` to get a new Tensor object.

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))